# JF image show average

In [1]:
import matplotlib
%matplotlib notebook
import os
import numpy as np
import scipy
from scipy import optimize, ndimage

import matplotlib.pyplot as plt
from matplotlib import cm
from extra_data import open_run
from extra_data import by_id
import pandas as pd
import zipfile
from PIL import Image
import shutil as shutil

In [2]:
propno = 2838
home_dir = "/home/vozdavoj/"
run = open_run(proposal=propno, run=134)
print(run)

<extra_data.DataCollection for 60 sources and 4025 trains>


## Prepare folder locations with experimental run and proposal number
## Change this cell!

In [7]:
# Create proposal directory
#propno = "3213" #"p002353" 
propno = "2838"
home_dir = "/home/vozdavoj/"
prop_dir = os.path.join(home_dir,"p"+propno)
if not os.path.exists(prop_dir): os.mkdir(prop_dir)
allruns_dir = os.path.join(prop_dir,"runs")
if not os.path.exists(allruns_dir): os.mkdir(allruns_dir)
#JF3_dir = run_dir + "/JNGFR03"
#JF2_dir = run_dir + "/JNGFR02"

os.chdir(home_dir)

#if not os.path.exists(JF3_dir):
#    os.mkdir(JF3_dir)

#if not os.path.exists(JF2_dir):
#    os.mkdir(JF2_dir)

print(os.getcwd())

/home/vozdavoj


### Open run and create run folder

In [ ]:
#Select data by JF3 signal and save
%matplotlib inline

# T = [0.02, 1.0, 0.43, 0.18]
#runs = [45,46,47,48]       #0fs
#runs = [50,51,52,53]       #25fs rather 25as
#runs = [56,58,59,60]       #50fs rather 50as
#runs = [56, 61]            # check pump/probe ratio after operators mistake
#runs = [63,64,65,66]       #75fs
#runs = [69,70,71,72]        #100fs
#runs = [75,76,77,79]        #125fs
#runs = [81,82,85,86]        #150fs
#runs = [88,89,90,91]        #175fs
#runs = [93,94,95,96]        #200fs
#runs = [99,100,101,102]     #225fs
#runs = [104,105,106,107]    #250fs
#runs = [109,110,111,112]    #300fs
#runs = [114,115,116,117]    #350fs
#runs = [119,120,121,122]    #25fs
#runs = [124,125,126,127]    #50fs
#runs = [128,129,130,131]    #75fs

runs = [134]                 #capton diffuse scattering calibration

# T = [0.02, 1.0, 0.75, 0.57, 0.32, 0.1]     # transmissions  
#runs = [135,136,137,138,139,140]    #0fs ...DONE
#runs = [142,143,144,145,146,147]    #25fs ...DONE
#runs = [148,149,150,151,152,153]    #50fs  ...DONE
#runs = range(154,160,1)             #75fs  ...DONE
#runs = range(160,166,1)             #100fs ...DONE
#runs = range(166,172,1)             #125fs ...DONE
#runs = range(172,178,1)             #150fs ...DONE
#runs = range(178,184,1)             #175fs ...DONE
#runs = range(185,191,1)             #200fs ...DONE
#runs = range(191,197,1)             #225fs ...DONE
#runs = range(197,203,1)             #250fs ...DONE
#runs = range(203,209,1)             #275fs ...DONE
#runs = range(209,215,1)              #300fs ...DONE
#runs = range(215,218,1)              #325fs acc. crash
#runs = range(218,224,1)              #325fs ...DONE
#runs = range(224,230,1)              #350fs ...DONE
#runs = range(230,236,1)              #375fs ...DONE
#runs = range(236,242,1)              #390fs ...DONE
#runs = [244]                         #alumina f-scan

# /INSTRUMENT/HED_IA1_EPX100-2/DET/RECEIVER:daqOutput/data/image/pixels
# /INSTRUMENT/HED_IA1_EPX100-1/DET/RECEIVER:daqOutput/data/image/pixels
# /INSTRUMENT/HED_IA1_JF500K3/DET/JNGFR03:daqOutput/data/adc
# /INSTRUMENT/HED_IA1_JF500K2/DET/JNGFR02:daqOutput/data/adc

# PPU = run['HED_XTD6_PPU/MDL/PPU_TRIGGER','trainTrigger.sequenceStart.value'].xarray().squeeze()
# SA2_XTD1_XGM_iTD = run['SA2_XTD1_XGM/XGM/DOOCS:output', 'data.intensityTD'].xarray().squeeze()
# SA2_XTD1_XGM_tID = run['SA2_XTD1_XGM/XGM/DOOCS:output', 'data.trainId'].xarray().squeeze()

# HED_XTD6_XGM_iTD = run['HED_XTD6_XGM/XGM/DOOCS:output', 'data.intensityTD'].xarray().squeeze()
# HED_XTD6_XGM_tID = run['HED_XTD6_XGM/XGM/DOOCS:output', 'data.trainId'].xarray().squeeze()

#runs = [135,142,148,154,160,166,172,178,185,191,197,203,209,218,224,230,236]
#runs = runs + np.ones(len(runs), dtype = int)

#runs = [18,20,21]

for i in range(0, len(runs)):
    #print(runs[i])
    run = open_run(propno, runs[i], "proc")
    print("Run " + str(runs[i]) + " contains " + str(len(run.train_ids)) + " train_IDs.")
    
    # Array for count,train_ids, JF3 sum, SA2 XGM, HED XGM, JF2 pump, JF2 probe
    sums = np.zeros([7, len(run.train_ids)])
    
    # Create run folder and subfolders
    run_dir = allruns_dir + "/Run" + str(runs[i])
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    
    JF3_dir = run_dir + "/JNGFR03"
    JF2_dir = run_dir + "/JNGFR02"
    
    if not os.path.exists(JF3_dir):
        os.mkdir(JF3_dir)

    if not os.path.exists(JF2_dir):
        os.mkdir(JF2_dir)
       
    
    # Save JF3 data and identify used trains
    print("Extracting JF3 XRD data ... ")
    os.chdir(JF3_dir)
    print("Saving JF3 data to:" + os.getcwd()) 
   
    end = len(run.train_ids)
    start = 0
    count = 0


    for j in range(start, end):
        sel = run.select_trains(j)
        JF3 = sel['HED_IA1_JF500K3/DET/JNGFR03:daqOutput', 'data.adc'].xarray().squeeze()       
                   
        # optimum threshold between 500000 and 1000000
        # put zero to get all images
        if JF3.data.sum().__gt__(0.0):
            count = count + 1
            
            f = open("T"+str(run.train_ids[j])+".dat", "w")
            JF3.data.tofile(f)
            f.close()
            
            sums[0,count-1] = count
            sums[1,count-1] = run.train_ids[j]
            sums[2,count-1] = JF3.data.sum()           
           
    print("Extracted images: " + str(count))

    sums = sums[:,0:count]                     # cut off useless ids
    used_ids = sums[1,:]    
    
    # Pulse energy data (XGM)
    print("Extracting XGM data ...")
    for k in range(0,count):
        SA2_XTD1_XGM_iTD = run['SA2_XTD1_XGM/XGM/DOOCS:output', 'data.intensityTD'][by_id[[int(used_ids[k])]]].xarray().squeeze()
        HED_XTD6_XGM_iTD = run['HED_XTD6_XGM/XGM/DOOCS:output', 'data.intensityTD'][by_id[[int(used_ids[k])]]].xarray().squeeze()
        if len(SA2_XTD1_XGM_iTD.data).__gt__(0):
            sums[3,k] = SA2_XTD1_XGM_iTD.data[0]
        else:
            sums[3,k] = 0
            
        if len(HED_XTD6_XGM_iTD.data).__gt__(0):
            sums[4,k] = HED_XTD6_XGM_iTD.data[0]
        else:
            sums[4,k] = 0
    
    # Save JF2 data and integrate peaks
    print("Extracting JF2 spectra data ...")
    os.chdir(JF2_dir)
    print("Saving JF2 data to:" + os.getcwd())  
    
    JF2 = run['HED_IA1_JF500K2/DET/JNGFR02:daqOutput', 'data.adc'][by_id[used_ids]].xarray().squeeze()
    for k in range(0,count):
        
        ## Full image
        spectrum = JF2.data[k,:,:]
        f_spectrum = open("T"+str(int(used_ids[k]))+".dat", "w")
        spectrum.tofile(f_spectrum)
        f_spectrum.close()   
        
        ## Cropped image
        #spectrum = np.where(JF2.data[k,:,:]>=0,JF2.data[k,:,:],0)
        #pump = spectrum[330:460,140:250]
        #probe = spectrum[330:460,250:335]
        #sums[5,k] = pump.sum()
        #sums[6,k] = probe.sum()
                
        #f_pump = open("T"+str(int(used_ids[k]))+"_pump.dat", "w")
        #pump.tofile(f_pump)
        #f_pump.close()
        
        #f_probe = open("T"+str(int(used_ids[k]))+"_probe.dat", "w")
        #probe.tofile(f_probe)
        #f_probe.close()
        
        #tif_pump = Image.fromarray(pump)
        #tif_pump.save("T"+str(int(used_ids[k]))+"_pump.tif")
        #tif_probe = Image.fromarray(probe)
        #tif_probe.save("T"+str(int(used_ids[k]))+"_probe.tif")

    # Save text data
    os.chdir(run_dir)
    print("Saving text data to: " + os.getcwd())
    print("")
    np.savetxt("Run" + str(runs[i])+ "_data.txt", np.transpose(sums), delimiter = ',', header = "Count, Train_ID, JF3.sum, SA2_XTD1_XGM [uJ], HED_XTD6_XGM [uJ], JF2.pump.sum, JF2.probe.sum") 
    
# Save the zip file
os.chdir(prop_dir)
print("Saving ZIP file to: " +os.getcwd())

name = "Runs"
for i in range(len(runs)):
    name = name + "_" + str(runs[i])
    
zf = zipfile.ZipFile(name + ".zip", "w")
for root, dirs, files in os.walk(allruns_dir):
    for directory in dirs:
        #print(directory)
        zf.write(os.path.join(root, directory))
    for file in files:
        #print(file)
        zf.write(os.path.join(root, file))
zf.close()
print("Done.")    
    
# Clean-up the run_dir for next run
print("Deleting files in: " + allruns_dir)
for root, dirs, files in os.walk(allruns_dir):
    for file in files:      
        if os.path.isfile(os.path.join(root, file)):
            #print("Deleting file: " + os.path.join(root, file))
            os.remove(os.path.join(root, file)) 
print("Done.")

print("Deleting folder: " + allruns_dir)
shutil.rmtree(allruns_dir)
print("Done.")
print("")


# Save tif better
#tifimage = Image.fromarray(average)
#tifimage.save("Run"+str(runs[i])+".tif")


In [ ]:
#Select data by PPU
#Single shot referenced data

runs = [43,49,62,68,74,80,87,92,97,103,108,118,123]
delays = [0,0,75,100,125,150,175,200,225,250,300,25,50]
          
for i in range(0, len(runs)):
    #print(runs[i])
    run = open_run(propno, runs[i], "proc")
    print("Run " + str(runs[i]) + " contains " + str(len(run.train_ids)) + " train_IDs.")
    
    # Create run folder and subfolders
    run_dir = allruns_dir + "/Run" + str(runs[i])+"_"+str(delays[i])+"fs"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    
    JF3_dir = run_dir + "/JNGFR03"
    JF2_dir = run_dir + "/JNGFR02"
    
    if not os.path.exists(JF3_dir):
        os.mkdir(JF3_dir)

    if not os.path.exists(JF2_dir):
        os.mkdir(JF2_dir)
   
    #TraiIDs used
    PPU = run['HED_XTD6_PPU/MDL/PPU_TRIGGER','trainTrigger.sequenceStart.value'].xarray().squeeze()
    PPU = PPU + 1
    
    # Save JF3 data using PPU
    print("Extracting JF3 XRD data ... ")
    os.chdir(JF3_dir)
    print("Saving JF3 data to:" + os.getcwd()) 

    JF3_select = run['HED_IA1_JF500K3/DET/JNGFR03:daqOutput', 'data.adc'][by_id[PPU.data]]
    
    # Array for count,train_ids, JF3 sum, SA2 XGM, HED XGM, JF2 pump, JF2 probe
    sums = np.zeros([7, len(JF3_select.train_ids)])
    
    end = len(JF3_select.train_ids)
    start = 0
    
    for j in range(start, end):       
        JF3 = JF3_select[j].xarray().squeeze()       

        f = open("T"+str(JF3_select.train_ids[j])+".dat", "w")
        JF3.data.tofile(f)
        f.close()

            
        sums[0,j] = j
        sums[1,j] = JF3_select.train_ids[j]
        sums[2,j] = JF3.data.sum()           
           
    print("Extracted images: " + str(j+1))
    
    used_ids = sums[1,:] 
    
    # Pulse energy data (XGM)
    print("Extracting XGM data ...")
    for k in range(start,end):
        SA2_XTD1_XGM_iTD = run['SA2_XTD1_XGM/XGM/DOOCS:output', 'data.intensityTD'][by_id[[int(used_ids[k])]]].xarray().squeeze()
        HED_XTD6_XGM_iTD = run['HED_XTD6_XGM/XGM/DOOCS:output', 'data.intensityTD'][by_id[[int(used_ids[k])]]].xarray().squeeze()
        if len(SA2_XTD1_XGM_iTD.data).__gt__(0):
            sums[3,k] = SA2_XTD1_XGM_iTD.data[0]
        else:
            sums[3,k] = 0
            
        if len(HED_XTD6_XGM_iTD.data).__gt__(0):
            sums[4,k] = HED_XTD6_XGM_iTD.data[0]
        else:
            sums[4,k] = 0
    
    # Save JF2 data and integrate peaks
    print("Extracting JF2 spectra data ...")
    os.chdir(JF2_dir)
    print("Saving JF2 data to:" + os.getcwd())  
    
    JF2 = run['HED_IA1_JF500K2/DET/JNGFR02:daqOutput', 'data.adc'][by_id[used_ids]].xarray().squeeze()
    for k in range(start,end):
        
        ## Full image
        spectrum = JF2.data[k,:,:]
        f_spectrum = open("T"+str(int(used_ids[k]))+".dat", "w")
        spectrum.tofile(f_spectrum)
        f_spectrum.close()   
        
        ## Cropped image
        #spectrum = np.where(JF2.data[k,:,:]>=0,JF2.data[k,:,:],0)
        #pump = spectrum[330:460,140:250]
        #probe = spectrum[330:460,250:335]
        #sums[5,k] = pump.sum()
        #sums[6,k] = probe.sum()
                
        #f_pump = open("T"+str(int(used_ids[k]))+"_pump.dat", "w")
        #pump.tofile(f_pump)
        #f_pump.close()
        
        #f_probe = open("T"+str(int(used_ids[k]))+"_probe.dat", "w")
        #probe.tofile(f_probe)
        #f_probe.close()
        
        #tif_pump = Image.fromarray(pump)
        #tif_pump.save("T"+str(int(used_ids[k]))+"_pump.tif")
        #tif_probe = Image.fromarray(probe)
        #tif_probe.save("T"+str(int(used_ids[k]))+"_probe.tif")

    # Save text data
    os.chdir(run_dir)
    print("Saving text data to: " + os.getcwd())
    print("")
    np.savetxt("Run" + str(runs[i])+ "_data.txt", np.transpose(sums), delimiter = ',', header = "Count, Train_ID, JF3.sum, SA2_XTD1_XGM [uJ], HED_XTD6_XGM [uJ], JF2.pump.sum, JF2.probe.sum") 
    
# Save the zip file
os.chdir(prop_dir)
print("Saving ZIP file to: " +os.getcwd())

name = "Runs"
for i in range(len(runs)):
    name = name + "_" + str(runs[i])
    
zf = zipfile.ZipFile(name + ".zip", "w")
for root, dirs, files in os.walk(allruns_dir):
    for directory in dirs:
        #print(directory)
        zf.write(os.path.join(root, directory))
    for file in files:
        #print(file)
        zf.write(os.path.join(root, file))
zf.close()
print("Done.")    
    
# Clean-up the run_dir for next run
print("Deleting files in: " + allruns_dir)
for root, dirs, files in os.walk(allruns_dir):
    for file in files:      
        if os.path.isfile(os.path.join(root, file)):
            #print("Deleting file: " + os.path.join(root, file))
            os.remove(os.path.join(root, file)) 
print("Done.")

print("Deleting folder: " + allruns_dir)
shutil.rmtree(allruns_dir)
print("Done.")
print("")